As the initial step, import the required libraries that we require to perform the tasks like preprocessing, training our model and predicting the accuracy score of our model

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,precision_score,recall_score,f1_score

Import the dataset to the simulation software and to view the first 5 rows of our dataset we used head function in pandas dataframe

In [2]:
df=pd.read_csv("/content/drive/MyDrive/aquaponics/IoTPond10.csv")
df.head()

,created_at,entry_id,TEMPERATURE,TURBIDITY,DISOLVED OXYGEN,pH,AMMONIA,NITRATE,Population,Length,Weight
0,2021-06-25 15:48:59 CET,1,-127.0000,100,0.0,15.45166,0.0,0,50,13.45,27.6
1,2021-06-25 15:49:19 CET,2,-127.0000,100,0.0,15.45166,0.0,0,50,13.45,27.6
2,2021-06-25 15:50:16 CET,3,27.1250,100,0.0,6.01411,0.0,139,50,13.45,27.6
3,2021-06-25 15:51:50 CET,4,27.3125,100,0.0,6.03227,0.0,131,50,13.45,27.6
4,2021-06-25 15:52:10 CET,5,27.2500,100,0.0,6.03681,0.0,133,50,13.45,27.6


In [3]:
df.describe()

,entry_id,TEMPERATURE,TURBIDITY,DISOLVED OXYGEN,pH,AMMONIA,NITRATE,Population,Length,Weight
count,620.000000,620.000000,620.000000,620.000000,620.000000,620.00000,620.000000,620.0,620.000000,620.000000
mean,310.500000,22.163508,40.361290,2.357571,5.657324,inf,311.558065,50.0,27.210543,282.574563
std,179.122863,27.271589,58.229381,4.398998,2.095430,NaN,391.184946,0.0,13.046585,281.105905
min,1.000000,-127.000000,-97.000000,0.000000,-1.975350,0.00000,0.000000,50.0,13.450000,27.600000
25%,155.750000,26.687500,-13.000000,0.000000,6.018650,0.00000,114.000000,50.0,15.571429,45.921394
50%,310.500000,27.125000,-9.000000,0.000000,6.041350,0.00001,193.000000,50.0,24.107143,158.532117
75%,465.250000,27.312500,100.000000,2.345000,6.163910,0.00522,229.000000,50.0,44.549290,689.142900
max,620.000000,31.687500,100.000000,32.411000,15.451660,inf,2473.000000,50.0,44.549290,689.142900


To check for the null values present in our dataset we use isnull() function


In [4]:
df.isnull().sum()

created_at         0
entry_id           0
TEMPERATURE        0
TURBIDITY          0
DISOLVED OXYGEN    0
pH                 0
AMMONIA            0
NITRATE            0
Population         0
Length             0
Weight             0
dtype: int64

Checking if the numbers are within limits are having infinite

In [5]:
df.isin([np.inf,-np.inf]).sum()

created_at         0
entry_id           0
TEMPERATURE        0
TURBIDITY          0
DISOLVED OXYGEN    0
pH                 0
AMMONIA            9
NITRATE            0
Population         0
Length             0
Weight             0
dtype: int64

Drop all those values that are infinite

In [6]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(axis=0,inplace=True)

Drop all the unnecessary columns from the dataset

In [7]:
df=df.drop(columns='entry_id')
df=df.drop(columns='Population')
df=df.drop(columns='created_at')

Check the datatype of each columns in our dataset

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 611 entries, 0 to 619
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   TEMPERATURE      611 non-null    float64
 1   TURBIDITY        611 non-null    int64  
 2   DISOLVED OXYGEN  611 non-null    float64
 3   pH               611 non-null    float64
 4   AMMONIA          611 non-null    float64
 5   NITRATE          611 non-null    int64  
 6   Length           611 non-null    float64
 7   Weight           611 non-null    float64
dtypes: float64(6), int64(2)
memory usage: 43.0 KB


We see that our classifications labels fish_length and fish_weight are in float variables and we need to convert them to string.

In [10]:
df=df.astype({"Length":'str',"Weight":'str'})

To find the corelation between the input variables

In [11]:
df.corr()

,TEMPERATURE,TURBIDITY,DISOLVED OXYGEN,pH,AMMONIA,NITRATE
TEMPERATURE,1.000000,0.117809,0.087587,-0.038160,0.010224,0.090179
TURBIDITY,0.117809,1.000000,-0.032924,0.347781,0.077227,-0.012663
DISOLVED OXYGEN,0.087587,-0.032924,1.000000,-0.225452,-0.014587,-0.158552
pH,-0.038160,0.347781,-0.225452,1.000000,0.037715,0.164342
AMMONIA,0.010224,0.077227,-0.014587,0.037715,1.000000,0.135740
NITRATE,0.090179,-0.012663,-0.158552,0.164342,0.135740,1.000000


In [12]:
x=df.drop(columns=['Length','Weight'])
y=df.drop(columns=['TEMPERATURE','TURBIDITY','DISOLVED OXYGEN','pH','AMMONIA','NITRATE'])
y1=y['Length']
y2=y['Weight']

Y1 corresponds to the model for fish length label and Y2 corresponds to the model for fish weight label. Splitting the data for training and testing


In [13]:
X_train,X_test,Y1_train,Y1_test=train_test_split(x,y1,test_size=0.2,random_state=0)
X_train,X_test,Y2_train,Y2_test=train_test_split(x,y2,test_size=0.2,random_state=0)

Saling of input data

In [14]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

1) Logistic Regression

In [15]:
model=LogisticRegression(multi_class='multinomial')
data=model.fit(X_train,Y1_train)
Y1_predict=data.predict(X_test)
print("The classification report for Fish Length is:",classification_report(Y1_test,Y1_predict))
print("The confusion matrix for Fish Length is:",confusion_matrix(Y1_test,Y1_predict))
data=model.fit(X_train,Y2_train)
Y2_predict=data.predict(X_test)
print("The classification report for Fish Weight is:",classification_report(Y2_test,Y2_predict))
print("The confusion matrix for Fish Weight is:",confusion_matrix(Y2_test,Y2_predict))

The classification report for Fish Length is:               precision    recall  f1-score   support

       13.45       0.41      0.28      0.33        25
 15.33571439       0.00      0.00      0.00         3
 15.57142869       0.00      0.00      0.00        19
 15.80714299       0.00      0.00      0.00         6
 16.04285729       0.00      0.00      0.00         2
 16.51428589       0.00      0.00      0.00         5
        31.7       0.33      0.58      0.42        24
    41.11143       0.00      0.00      0.00         1
    41.48786       0.00      0.00      0.00         7
    44.54929       0.48      0.94      0.63        31

    accuracy                           0.41       123
   macro avg       0.12      0.18      0.14       123
weighted avg       0.27      0.41      0.31       123

The confusion matrix for Fish Length is: [[ 7  0  0  0  0  0 18  0  0  0]
 [ 0  0  0  0  0  1  2  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 19]
 [ 0  0  0  0  0  0  0  0  0  6]
 [ 0  0  0  0  0  0  0

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with n

2) Decision Tree Classifier

In [16]:
model=DecisionTreeClassifier()
data=model.fit(X_train,Y1_train)
Y1_predict=data.predict(X_test)
print("The classification report for Fish Length is:",classification_report(Y1_test,Y1_predict))
print("The confusion matrix for Fish Length is:",confusion_matrix(Y1_test,Y1_predict))
data=model.fit(X_train,Y2_train)
Y2_predict=data.predict(X_test)
print("The classification report for Fish Weight is:",classification_report(Y2_test,Y2_predict))
print("The confusion matrix for Fish Weight is:",confusion_matrix(Y2_test,Y2_predict))

The classification report for Fish Length is:               precision    recall  f1-score   support

       13.45       0.08      0.08      0.08        25
 14.15714289       0.00      0.00      0.00         0
 15.10000009       0.00      0.00      0.00         0
 15.33571439       0.00      0.00      0.00         3
 15.57142869       0.19      0.21      0.20        19
 15.80714299       0.25      0.17      0.20         6
 16.04285729       0.00      0.00      0.00         2
 16.27857159       0.00      0.00      0.00         0
 16.51428589       0.27      0.60      0.37         5
        31.7       0.04      0.04      0.04        24
    41.11143       0.00      0.00      0.00         1
    41.48786       0.00      0.00      0.00         7
    44.54929       0.08      0.06      0.07        31

    accuracy                           0.11       123
   macro avg       0.07      0.09      0.07       123
weighted avg       0.10      0.11      0.10       123

The confusion matrix for Fish Len

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1

3) Support Vector Machine

In [17]:
model=SVC()
data=model.fit(X_train,Y1_train)
Y1_predict=data.predict(X_test)
print("The classification report for Fish Length is:",classification_report(Y1_test,Y1_predict))
print("The confusion matrix for Fish Length is:",confusion_matrix(Y1_test,Y1_predict))
data=model.fit(X_train,Y2_train)
Y2_predict=data.predict(X_test)
print("The classification report for Fish Weight is:",classification_report(Y2_test,Y2_predict))
print("The confusion matrix for Fish Weight is:",confusion_matrix(Y2_test,Y2_predict))

The classification report for Fish Length is:               precision    recall  f1-score   support

       13.45       0.00      0.00      0.00        25
 15.33571439       0.20      0.67      0.31         3
 15.57142869       0.00      0.00      0.00        19
 15.80714299       0.00      0.00      0.00         6
 16.04285729       0.00      0.00      0.00         2
 16.27857159       0.00      0.00      0.00         0
 16.51428589       0.00      0.00      0.00         5
        31.7       0.47      1.00      0.64        24
    41.11143       0.00      0.00      0.00         1
    41.48786       0.00      0.00      0.00         7
    44.54929       0.47      0.90      0.62        31

    accuracy                           0.44       123
   macro avg       0.10      0.23      0.14       123
weighted avg       0.21      0.44      0.29       123

The confusion matrix for Fish Length is: [[ 0  0  0  0  0  0  0 25  0  0  0]
 [ 0  2  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1

4) Random Forest Classifier

In [18]:
model=RandomForestClassifier()
data=model.fit(X_train,Y1_train)
Y1_predict=data.predict(X_test)
print("The classification report for Fish Length is:",classification_report(Y1_test,Y1_predict))
print("The confusion matrix for Fish Length is:",confusion_matrix(Y1_test,Y1_predict))
data=model.fit(X_train,Y2_train)
Y2_predict=data.predict(X_test)
print("The classification report for Fish Weight is:",classification_report(Y2_test,Y2_predict))
print("The confusion matrix for Fish Weight is:",confusion_matrix(Y2_test,Y2_predict))

The classification report for Fish Length is:               precision    recall  f1-score   support

       13.45       0.04      0.04      0.04        25
 14.15714289       0.00      0.00      0.00         0
 15.10000009       0.00      0.00      0.00         0
 15.33571439       0.00      0.00      0.00         3
 15.57142869       0.16      0.16      0.16        19
 15.80714299       0.00      0.00      0.00         6
 16.04285729       0.00      0.00      0.00         2
 16.27857159       0.00      0.00      0.00         0
 16.51428589       0.14      0.20      0.17         5
        31.7       0.08      0.08      0.08        24
    41.11143       0.00      0.00      0.00         1
    41.48786       0.00      0.00      0.00         7
    44.54929       0.18      0.19      0.18        31

    accuracy                           0.11       123
   macro avg       0.05      0.05      0.05       123
weighted avg       0.10      0.11      0.10       123

The confusion matrix for Fish Len

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1

5) K Nearest Neighbors

In [19]:
model=KNeighborsClassifier()
data=model.fit(X_train,Y1_train)
Y1_predict=data.predict(X_test)
print("The classification report for Fish Length is:",classification_report(Y1_test,Y1_predict))
print("The confusion matrix for Fish Length is:",confusion_matrix(Y1_test,Y1_predict))
data=model.fit(X_train,Y2_train)
Y2_predict=data.predict(X_test)
print("The classification report for Fish Weight is:",classification_report(Y2_test,Y2_predict))
print("The confusion matrix for Fish Weight is:",confusion_matrix(Y2_test,Y2_predict))

The classification report for Fish Length is:               precision    recall  f1-score   support

       13.45       0.15      0.16      0.16        25
 15.33571439       0.00      0.00      0.00         3
 15.57142869       0.21      0.21      0.21        19
 15.80714299       0.00      0.00      0.00         6
 16.04285729       0.00      0.00      0.00         2
 16.27857159       0.00      0.00      0.00         0
 16.51428589       0.12      0.20      0.15         5
        31.7       0.13      0.12      0.13        24
    41.11143       1.00      1.00      1.00         1
    41.48786       0.00      0.00      0.00         7
    44.54929       0.21      0.23      0.22        31

    accuracy                           0.16       123
   macro avg       0.17      0.17      0.17       123
weighted avg       0.15      0.16      0.16       123

The confusion matrix for Fish Length is: [[ 4  0  0  0  0  0  1 20  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  2  0]
 [ 0  0  4  0  0  0  0  0  0

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1

6) Ada Boost Classifier

In [20]:
model=AdaBoostClassifier()
data=model.fit(X_train,Y1_train)
Y1_predict=data.predict(X_test)
print("The classification report for Fish Length is:",classification_report(Y1_test,Y1_predict))
print("The confusion matrix for Fish Length is:",confusion_matrix(Y1_test,Y1_predict))
data=model.fit(X_train,Y2_train)
Y2_predict=data.predict(X_test)
print("The classification report for Fish Weight is:",classification_report(Y2_test,Y2_predict))
print("The confusion matrix for Fish Weight is:",confusion_matrix(Y2_test,Y2_predict))

The classification report for Fish Length is:               precision    recall  f1-score   support

       13.45       0.00      0.00      0.00        25
 14.15714289       0.00      0.00      0.00         0
 15.33571439       0.00      0.00      0.00         3
 15.57142869       0.00      0.00      0.00        19
 15.80714299       0.00      0.00      0.00         6
 16.04285729       0.00      0.00      0.00         2
 16.51428589       0.00      0.00      0.00         5
        31.7       0.41      1.00      0.58        24
    41.11143       0.00      0.00      0.00         1
    41.48786       0.00      0.00      0.00         7
    44.54929       0.49      1.00      0.66        31

    accuracy                           0.45       123
   macro avg       0.08      0.18      0.11       123
weighted avg       0.20      0.45      0.28       123

The confusion matrix for Fish Length is: [[ 0  0  0  0  0  0  0 25  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  3  0

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1

7) Extra Trees Classifier

In [21]:
model=ExtraTreesClassifier()
data=model.fit(X_train,Y1_train)
Y1_predict=data.predict(X_test)
print("The classification report for Fish Length is:",classification_report(Y1_test,Y1_predict))
print("The confusion matrix for Fish Length is:",confusion_matrix(Y1_test,Y1_predict))
data=model.fit(X_train,Y2_train)
Y2_predict=data.predict(X_test)
print("The classification report for Fish Weight is:",classification_report(Y2_test,Y2_predict))
print("The confusion matrix for Fish Weight is:",confusion_matrix(Y2_test,Y2_predict))

The classification report for Fish Length is:               precision    recall  f1-score   support

       13.45       0.04      0.04      0.04        25
 14.15714289       0.00      0.00      0.00         0
 15.10000009       0.00      0.00      0.00         0
 15.33571439       0.00      0.00      0.00         3
 15.57142869       0.16      0.16      0.16        19
 15.80714299       0.00      0.00      0.00         6
 16.04285729       0.00      0.00      0.00         2
 16.27857159       0.00      0.00      0.00         0
 16.51428589       0.14      0.20      0.17         5
        31.7       0.08      0.08      0.08        24
    41.11143       0.00      0.00      0.00         1
    41.48786       0.00      0.00      0.00         7
    44.54929       0.18      0.19      0.18        31

    accuracy                           0.11       123
   macro avg       0.05      0.05      0.05       123
weighted avg       0.10      0.11      0.10       123

The confusion matrix for Fish Len

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1

8) Gradient Boosting Classifier

In [22]:
model=GradientBoostingClassifier()
data=model.fit(X_train,Y1_train)
Y1_predict=data.predict(X_test)
print("The classification report for Fish Length is:",classification_report(Y1_test,Y1_predict))
print("The confusion matrix for Fish Length is:",confusion_matrix(Y1_test,Y1_predict))
data=model.fit(X_train,Y2_train)
Y2_predict=data.predict(X_test)
print("The classification report for Fish Weight is:",classification_report(Y2_test,Y2_predict))
print("The confusion matrix for Fish Weight is:",confusion_matrix(Y2_test,Y2_predict))

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1

The classification report for Fish Length is:               precision    recall  f1-score   support

       13.45       0.04      0.04      0.04        25
 14.15714289       0.00      0.00      0.00         0
 15.10000009       0.00      0.00      0.00         0
 15.33571439       0.00      0.00      0.00         3
 15.57142869       0.16      0.16      0.16        19
 15.80714299       0.25      0.17      0.20         6
 16.04285729       0.00      0.00      0.00         2
 16.27857159       0.00      0.00      0.00         0
 16.51428589       0.00      0.00      0.00         5
        31.7       0.08      0.08      0.08        24
    41.11143       0.00      0.00      0.00         1
    41.48786       0.00      0.00      0.00         7
    44.54929       0.18      0.19      0.18        31

    accuracy                           0.11       123
   macro avg       0.05      0.05      0.05       123
weighted avg       0.10      0.11      0.10       123

The confusion matrix for Fish Len

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1

9) Naive Bayes Classifier

In [23]:
model=MultinomialNB()
data=model.fit(X_train,Y1_train)
Y1_predict=data.predict(X_test)
print("The classification report for Fish Length is:",classification_report(Y1_test,Y1_predict))
print("The confusion matrix for Fish Length is:",confusion_matrix(Y1_test,Y1_predict))
data=model.fit(X_train,Y2_train)
Y2_predict=data.predict(X_test)
print("The classification report for Fish Weight is:",classification_report(Y2_test,Y2_predict))
print("The confusion matrix for Fish Weight is:",confusion_matrix(Y2_test,Y2_predict))

The classification report for Fish Length is:               precision    recall  f1-score   support

       13.45       0.00      0.00      0.00        25
 15.33571439       0.00      0.00      0.00         3
 15.57142869       0.00      0.00      0.00        19
 15.80714299       0.00      0.00      0.00         6
 16.04285729       0.00      0.00      0.00         2
 16.51428589       0.00      0.00      0.00         5
        31.7       0.00      0.00      0.00        24
    41.11143       0.00      0.00      0.00         1
    41.48786       0.00      0.00      0.00         7
    44.54929       0.24      0.94      0.38        31

    accuracy                           0.24       123
   macro avg       0.02      0.09      0.04       123
weighted avg       0.06      0.24      0.10       123

The confusion matrix for Fish Length is: [[ 0  0  0  0  0  0  1  0  0 24]
 [ 0  0  0  0  0  0  0  0  0  3]
 [ 0  0  0  0  0  0  0  0  0 19]
 [ 0  0  0  0  0  0  0  0  0  6]
 [ 0  0  0  0  0  0  0

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classificat